##### -*- coding: utf8 -*-
### Information
- Date : 2024-03-18 11:00
- Author : Okrie
- Description :   Airplane Price Information Test
- Version : 0.1

In [283]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

Skyscanner - https://partners.api.skyscanner.net/apiservices/

#### Skyscanner Info     
- seoul all , bk all
- round trip
- depart date = 2024.03.22
- arrival date = 2024.03.27
- adult = 2
- children = 2 ( 4, 10 )      
    
```code
https://www.skyscanner.co.kr/transport/flights/    
sela/                           departure    
bkkt/                           arrival    
240322/                         departure_date    
240327/                         arrival_date    
?adults=2                       adult_count    
&adultsv2=2                     ?           
&cabinclass=economy             carbin_class        
&children=1                     children_count      
&childrenv2=4%7c10              children_type 4|10                 
&inboundaltsenabled=false       ?       
&infants=0                      영아=0 (0~1) 1~3살      
&outboundaltsenabled=false      ?       
&preferdirects=false            ?        
&ref=home                       ?       
&rtn=1                          ?       
&stops=!oneStop,!twoPlusStops   1회 경유, 2회 이상 경유 제외
```   
https://www.skyscanner.co.kr/transport/flights/icn/bkk/240322/240327/?adults=2&adultsv2=2&cabinclass=economy&children=2&childrenv2=4%7c10&destinationentityid=27536671&inboundaltsenabled=false&infants=0&originentityid=27538638&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1&stops=!oneStop,!twoPlusStops

In [2]:
departure = "ICN"
arrival = "NRT"

departure_date = "2024-04-02"
arrival_date = "2024-04-05"

adults_count = 1
children_count = 0

cabinclass = "economy"

Convert Date    
2024-03-01 => 240301

```python
    if person_count == 1:
        "adultsv2=2"
    else :
        "adults=2"
        "adultsv2=2"
```

https://www.skyscanner.co.kr/transport/flights/icn/bkk/240322/240327/?adults=2&adultsv2=2&cabinclass=economy&children=2&childrenv2=4%7c10&destinationentityid=27536671&inboundaltsenabled=false&infants=0&originentityid=27538638&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1&stops=!oneStop,!twoPlusStops

In [3]:
def convertDateTouseUrl(date: str):
    return date.replace("-", "")[2:]

In [4]:
def hasChildrenCheck(children):
    if children > 0:
        return True
    return False

def childrenAgeCheck(*args):
    tmp = args
    returnString = f'&children={children_count}&childrenv2='
    addSeperateString = '%7c'
    
    for value in tmp:
        returnString += str(value)
        returnString += addSeperateString
        
    if returnString[-1] == 'c':
        returnString = returnString[:-len(addSeperateString)]
    return returnString

In [5]:
url = f'https://www.skyscanner.co.kr/transport/flights/'
url += departure.lower() + "/"
url += arrival.lower() + "/"
url += convertDateTouseUrl(departure_date) + "/"
url += convertDateTouseUrl(arrival_date) + "/"
url += f'?adults={adults_count}&adultsv2={adults_count}'
url += f'&cabinclass={cabinclass}'
url += childrenAgeCheck(0) if hasChildrenCheck(children_count) else ''
url += f'&destinationentityid=27536671&inboundaltsenabled=false'
url += f'&infants=0'
url += f'&originentityid=27538638&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1'
url += f'&stops=!oneStop,!twoPlusStops'                                     # 직항만

In [10]:
print(url)

https://www.skyscanner.co.kr/transport/flights/icn/nrt/240402/240405/?adults=1&adultsv2=1&cabinclass=economy&destinationentityid=27536671&inboundaltsenabled=false&infants=0&originentityid=27538638&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1&stops=!oneStop,!twoPlusStops


In [11]:
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(30)

In [101]:
driver.delete_all_cookies()
html = driver.page_source
soup = BeautifulSoup(html)

#dayview-first-result > div:nth-child(1)

In [341]:
skyscanner = soup.select('div.FlightsResults_dayViewItems__ZDFlO > div > div > div > a')

In [357]:
columns = [
    'site',
    'dp_departure_airline',
    'dp_departure_img',
    'dp_departure_time',
    'dp_arrival_time',
    'dp_duration',
    'dp_departure_airport',
    'dp_arrival_airport',
    'dp_transit',
    'entry_departure_airline',
    'entry_departure_img',
    'entry_departure_time',
    'entry_arrival_time',
    'entry_duration',
    'entry_departure_airport',
    'entry_arrival_airport',
    'entry_transit',
    'ticket_info',
    'ticket_price',
    'link',
]

df = pd.DataFrame(columns=columns)

In [358]:
site = "skyscanner"

for index, value in enumerate(skyscanner):
    one_data = {}
    
    # 출국
    dp_departure_airline = value.select('div')[0].span.getText()
    
    dp_departure_img = f"https://{value.select('div.LegLogo_legImage__MmY0Z')[0].img.attrs['src'][2:]}"
    
    dp_departure_time = value.select('div.LegInfo_routePartialDepart__NzEwY')[0].span.get_text()
    dp_arrival_time = value.select('div.LegInfo_routePartialArrive__Y2U1N')[0].span.get_text()
    
    dp_duration = value.select('div.LegInfo_stopsContainer__NWIyN')[0].span.get_text()
    
    dp_departure_airport = value.select('div.LegInfo_routePartialDepart__NzEwY > span:nth-child(2)')[0].span.get_text()
    dp_arrival_airport = value.select('div.LegInfo_routePartialArrive__Y2U1N > span:nth-child(2)')[0].span.get_text()
    
    dp_transit = value.select('div.LegInfo_stopsLabelContainer__MmM0Z')[0].span.get_text()
    
    # 입국
    entry_departure_airline = value.select('div')[1].span.getText()
    
    entry_departure_img  = f"https://{value.select('div.LegLogo_legImage__MmY0Z')[1].img.attrs['src'][2:]}"
    
    entry_departure_time = value.select('div.LegInfo_routePartialDepart__NzEwY')[1].span.get_text()
    entry_arrival_time = value.select('div.LegInfo_routePartialArrive__Y2U1N')[1].span.get_text()
    
    entry_duration = value.select('div.LegInfo_stopsContainer__NWIyN')[1].span.get_text()
    
    entry_departure_airport = value.select('div.LegInfo_routePartialDepart__NzEwY > span:nth-child(2)')[1].span.get_text()
    entry_arrival_airport = value.select('div.LegInfo_routePartialArrive__Y2U1N > span:nth-child(2)')[1].span.get_text()
    
    entry_transit = value.select('div.LegInfo_stopsLabelContainer__MmM0Z')[1].span.get_text()
    
    # Ticket 정보 및 가격
    ticket_info = value.select('div.TicketStub_horizontalStubContainer__YjM5M')[0].span.get_text()
    ticket_price = value.select('div.Price_mainPriceContainer__MDM3O')[0].span.get_text()
    
    link = f"https://www.skyscanner.co.kr{skyscanner[index].attrs['href']}"
    
    one_data = {
        'site' : site,
        'dp_departure_airline' : dp_departure_airline,
        'dp_departure_img' : dp_departure_img,
        'dp_departure_time' : dp_departure_time,
        'dp_arrival_time' : dp_arrival_time,
        'dp_duration' : dp_duration,
        'dp_departure_airport' : dp_departure_airport,
        'dp_arrival_airport' : dp_arrival_airport,
        'dp_transit' : dp_transit,
        'entry_departure_airline' : entry_departure_airline,
        'entry_departure_img' : entry_departure_img,
        'entry_departure_time' : entry_departure_time,
        'entry_arrival_time' : entry_arrival_time,
        'entry_duration' : entry_duration,
        'entry_departure_airport' : entry_departure_airport,
        'entry_arrival_airport' : entry_arrival_airport,
        'entry_transit' : entry_transit,
        'ticket_info' : ticket_info,
        'ticket_price' : ticket_price,
        'link' : link,
    }
    
    df = pd.concat([df, pd.DataFrame([one_data])], ignore_index=True)


In [359]:
df

,site,dp_departure_airline,dp_departure_img,dp_departure_time,dp_arrival_time,dp_duration,dp_departure_airport,dp_arrival_airport,dp_transit,entry_departure_airline,entry_departure_img,entry_departure_time,entry_arrival_time,entry_duration,entry_departure_airport,entry_arrival_airport,entry_transit,ticket_info,ticket_price,link
0,skyscanner,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 8:00,오전 10:30,2시간 30분,ICN,NRT,직항,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오후 6:30,오후 9:15,2시간 45분,NRT,ICN,직항,총 18건 중 최저가,"₩297,400",https://www.skyscanner.co.kr/transport/flights...
1,skyscanner,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 7:05,오전 9:50,2시간 45분,ICN,NRT,직항,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오후 6:30,오후 9:15,2시간 45분,NRT,ICN,직항,총 18건 중 최저가,"₩309,200",https://www.skyscanner.co.kr/transport/flights...
2,skyscanner,진에어,https://www.skyscanner.net/images/airlines/sma...,오전 9:45,오후 12:15,2시간 30분,ICN,NRT,직항,진에어,https://www.skyscanner.net/images/airlines/sma...,오후 1:25,오후 3:45,2시간 20분,NRT,ICN,직항,총 21건 중 최저가,"₩316,700",https://www.skyscanner.co.kr/transport/flights...
3,skyscanner,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 8:00,오전 10:30,2시간 30분,ICN,NRT,직항,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 11:30,오후 2:35,3시간 05분,NRT,ICN,직항,총 18건 중 최저가,"₩281,700",https://www.skyscanner.co.kr/transport/flights...
4,skyscanner,진에어,https://www.skyscanner.net/images/airlines/sma...,오전 6:45,오전 9:20,2시간 35분,ICN,NRT,직항,진에어,https://www.skyscanner.net/images/airlines/sma...,오후 1:25,오후 3:45,2시간 20분,NRT,ICN,직항,총 21건 중 최저가,"₩316,600",https://www.skyscanner.co.kr/transport/flights...
5,skyscanner,진에어,https://www.skyscanner.net/images/airlines/sma...,오전 7:25,오전 10:00,2시간 35분,ICN,NRT,직항,진에어,https://www.skyscanner.net/images/airlines/sma...,오후 1:25,오후 3:45,2시간 20분,NRT,ICN,직항,총 21건 중 최저가,"₩329,100",https://www.skyscanner.co.kr/transport/flights...
6,skyscanner,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 7:05,오전 9:50,2시간 45분,ICN,NRT,직항,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오전 11:30,오후 2:35,3시간 05분,NRT,ICN,직항,총 18건 중 최저가,"₩293,500",https://www.skyscanner.co.kr/transport/flights...
7,skyscanner,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오후 3:10,오후 5:30,2시간 20분,ICN,NRT,직항,이스타항공,https://www.skyscanner.net/images/airlines/sma...,오후 6:30,오후 9:15,2시간 45분,NRT,ICN,직항,총 18건 중 최저가,"₩322,000",https://www.skyscanner.co.kr/transport/flights...


In [362]:
df.to_csv('test.csv', encoding='utf-8-sig')